### Motion Saliency Segmentation using 3D Gray-Code Kernels
#### Elena Nicora, MaLGa Centre, UniGe (2022) 
-----------------------------------------------------------------------------------------
This code implements the entire **Attentive Module** of 

> "On the use of efficient projection kernels for motion-based visual saliency estimation."
Nicora, Elena, and Nicoletta Noceti. Frontiers in Computer Science: 67.

The algorithm requires in input a pair of maps, max_pooling and avg_pooling, and returns a refined segmentation of the moving object in the scene.

**IMPORTANT:** Pooling maps required in input are computed using __[3D Gray-Code Kernels](https://github.com/Malga-Vision/3DGrayCodeKernels)__ (see paper cited above for more information)

For each pair of pooling maps we can identify the following steps:
1. **Otsu's adaptive thresholding**: returns a coarse segmentation (GCKsA) based only on max_pooling
1. Refinement of GCKsA using **minima and maxima** of the avg_pooling: here we discard connected components of GCKsA smaller than a 5x5 patch and those that do not correspond to areas where there are local extrema
1. **Identification of "present" and "past"** values by using two thresholds (delta_1 and delta_2) over the avg_pooling
1. Composition of the **refined segmentation (GCKsR)** subtracting the pixels corresponding to the "past" and adding the "present" pixels to consolidate the final result.

Note that every step contains several morphological operations in order to produce a visually pleasing segmentation with the least number of holes and artifacts.

Users can tune variables **delta_1 and delta_2**.

We assume that values or the avg_pooling near zero correspond to the past positions and values near 1 correspond to the present. But notice that in some cases it is the opposite so the user needs to invert the name of the variables in first_ref (first refinement that subtracts the "past") and second_ref (second refinement that adds the "present").



In [1]:
import glob
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
import cv2
from skimage.feature import peak_local_max
import time

In [2]:
### to be used on max pooling in order to obtain a first rough segmentation
### based on spatio-temporal information

def otsuAdaptiveThreshold(pooling_map):
    gauss_max = cv2.GaussianBlur(pooling_map,(5,5),0)
    
    autothr, binary_gauss = cv2.threshold(gauss_max, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    binary_gauss = cv2.dilate(binary_gauss, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)))
    binary_gauss = cv2.morphologyEx(binary_gauss, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(4,4)))
    binary_gauss = ndimage.binary_fill_holes(binary_gauss)
    binary_gauss = binary_gauss*255
    binary_gauss = binary_gauss.astype(np.uint8)
    
    return binary_gauss

In [3]:
### to be used on avg pooling
def maskPooling(pooling_map, mask):
    masked_pooling = np.zeros_like(pooling_map)
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i,j]==255:
                masked_pooling[i,j] = pooling_map[i,j]
    
    return masked_pooling

In [4]:
### to be used on avg_pooling

def findPeaks(pooling_map):
    if np.max(pooling_map)==1:
        pooling_map = pooling_map*255
        pooling_map = pooling_map.astype(np.uint8)
        
    thr_max = np.max(pooling_map)*80/100
    
    ## finds top 10 local maxima above the background interval
    max_peaks = peak_local_max(pooling_map, indices=True,  exclude_border = True, threshold_abs = thr_max, min_distance=5)
    if max_peaks.shape[0]>10:
            limit = 10
    else:
        limit = max_peaks.shape[0]
    
    maxima = np.zeros((limit,3))
    if max_peaks.shape[0]>0:
        for i in range(limit):
            maxima[i,0] = int(max_peaks[i,0])
            maxima[i,1] = int(max_peaks[i,1])
            maxima[i,2] = pooling_map[int(max_peaks[i,0]), int(max_peaks[i,1])]
    
    ## finds top 10 local minima below the background interval
    thr_min = np.max(cv2.bitwise_not(pooling_map)) * 80 /100
    inv = cv2.bitwise_not(pooling_map)
    min_peaks = peak_local_max(inv, indices=True, exclude_border = True, threshold_abs = thr_min, min_distance=5)

    minima = np.zeros((min_peaks.shape[0],3))
    if min_peaks.shape[0]>0:
        if min_peaks.shape[0]>10:
            limit = 10
        else:
            limit = min_peaks.shape[0]
        for i in range(limit):
            minima[i,0] = int(min_peaks[i,0])
            minima[i,1] = int(min_peaks[i,1])
            minima[i,2] = pooling_map[int(min_peaks[i,0]), int(min_peaks[i,1])]

    return minima, maxima
    

In [5]:
### delete connComp of area lower than 25 pixels (approx 5x5 patches)
def cleanConnComp(binary_mask):
    cc_stats = cv2.connectedComponentsWithStats(binary_mask, 4, cv2.CV_32S)
    num_cc = cc_stats[0]
    labels = cc_stats[1]
    
    centroids = cc_stats[3]
    connComp = np.copy(binary_mask)
    stats = cc_stats[2]
    
    for i in range(1,stats.shape[0]):
        count = 0
        if stats[i,4]<=30:     
            count=count+1
            [x,y] = np.where(labels==i)
            connComp[x,y] = 0

    return connComp

In [6]:
### we discard connected components that do not overlap with areas containing local extrema
def refineUsingPeaks(pooling_map, minima, maxima):
    
    cc_stats = cv2.connectedComponentsWithStats(pooling_map, 4, cv2.CV_32S)
    num_cc = cc_stats[0]
    labels = cc_stats[1]
    
    centroids = cc_stats[3]
    connComp = np.zeros_like(pooling_map)
#     print(connComp.shape)
    stats = cc_stats[2]
    for i in range(1,stats.shape[0]):
        points = 0
        if stats[i,4]>25:     
            for m in range(len(minima)):
                x = int(minima[m,0])
                y = int(minima[m,1])
                if y >= stats[i,0] and y <= (stats[i,0]+ stats[i,2]) and x >= stats[i,1] and x <= (stats[i,1] + stats[i,3]):
                    points = points+1
            for m in range(len(maxima)):
                x = int(maxima[m,0])
                y = int(maxima[m,1])
                if y >= stats[i,0] and y <= (stats[i,0]+ stats[i,2]) and x >= stats[i,1] and x <= (stats[i,1] + stats[i,3]):
                    points = points+1
        if points>0:
            [x,y] = np.where(labels==i)
            connComp[x,y] = 255    
    
    return connComp

#### Main Function

In [22]:
folder = '/home/elenina/Desktop/gck/teatro_della_tosse/video4'
# savepath = ###
mean_exec = []

path_max = sorted(glob.glob(''.join([folder, '/st_max*.npy'])))
path_avg = sorted(glob.glob(''.join([folder, '/t_avg*.npy'])))

delta_1 = 0.2
delta_2 = 0.8

start = time.time()
for frame in range(len(path_max)):
    max_pooling = np.load(path_max[frame])
    max_pooling = max_pooling * 255
    max_pooling = max_pooling.astype(np.uint8)
    avg_pooling = np.load(path_avg[frame])

    start_coarse = time.time()
    binary_max = otsuAdaptiveThreshold(max_pooling)
    binary_max = cleanConnComp(binary_max.astype(np.uint8))
#     print('Execution time coarse segmentation: ', time.time() - start_coarse)
    start_ref = time.time()
    minima, maxima = findPeaks(avg_pooling)
    connComp = refineUsingPeaks(binary_max, minima, maxima)
    
    ### avg_pooling values are included in the range [0,1] with background values around 0.5
    
    ### we are going to isolate the part of the map near 1 and create a binary mask of it
    segm_pos = np.zeros((connComp.shape[0], connComp.shape[1]))
    segm_x, segm_y = np.where(avg_pooling>delta_2)
    segm_pos[segm_x,segm_y] = 255
    cleanedPos = cleanConnComp(segm_pos.astype(np.uint8))
    cleanedPos = cv2.dilate(cleanedPos, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)))
    cleanedPos = cv2.morphologyEx(cleanedPos, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)))
        
    ### in the same way we isolate the values near 0 and create a binary mask of them
    segm_neg = np.zeros((connComp.shape[0], connComp.shape[1]))
    segm_x, segm_y = np.where(avg_pooling<delta_1)
    segm_neg[segm_x,segm_y] = 255
    cleanedNeg = cleanConnComp(segm_neg.astype(np.uint8))
    cleanedNeg = cv2.dilate(cleanedNeg, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)))
    cleanedNeg = cv2.morphologyEx(cleanedNeg, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)))

    ### if we assume that near zero values corresponds to the "past positions" of the object
    ### then we are going to subtract them to the previously refined mask (connComp)
    first_ref = np.copy(connComp)
    first_ref[cleanedNeg==255]=0
    first_ref = cv2.morphologyEx(first_ref, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6)))
    
    ### in the same way we consolidate the mask adding the pixels of the "present"
    second_ref = cv2.bitwise_or(first_ref.astype(np.uint8),cleanedPos.astype(np.uint8))
    refined = second_ref
    refined = cv2.morphologyEx(refined, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)))
    refined = cv2.morphologyEx(refined, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)))

    im_out = ndimage.binary_fill_holes(refined)
    im_out = im_out*255
    im_out = im_out.astype(np.uint8)

    cc_stats = cv2.connectedComponentsWithStats(im_out, 4, cv2.CV_32S)
    num_cc = cc_stats[0]
    labels = cc_stats[1]
    centroids = cc_stats[3]
    stats = cc_stats[2]
    for i in range(1,stats.shape[0]):
        points = 0
        if stats[i,4]<=30:
            [x,y] = np.where(labels==i)
            im_out[x,y] = 0
    
#     plt.imshow(im_out)
#     plt.show()

    mean_exec.append(time.time()-start_coarse)

print(time.time()-start)
print(np.mean(mean_exec))

66.53295969963074
0.031430193083080836
